In [3]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import scipy.io as sio
from scipy.signal import resample
import os

import numpy as np
import random
import pickle 
from sklearn import preprocessing

%matplotlib inline
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

from models.EncDec import EncDecBiLSTMSkipNet

import operator


In [4]:
model = EncDecBiLSTMSkipNet(3,1)

In [5]:
cnn1_w, cnn1_b = model.cnn1.weight, model.cnn1.bias
cnn2_w, cnn2_b = model.cnn2.weight, model.cnn2.bias
cnn3_w, cnn3_b = model.cnn3.weight, model.cnn3.bias
cnn4_w, cnn4_b = model.cnn4.weight, model.cnn4.bias

dcnn1_w, dcnn1_b = model.dcnn1.weight, model.dcnn1.bias
dcnn2_w, dcnn2_b = model.dcnn2.weight, model.dcnn2.bias
dcnn3_w, dcnn3_b = model.dcnn3.weight, model.dcnn3.bias
dcnn4_w, dcnn4_b = model.dcnn4.weight, model.dcnn4.bias

rnn1_w, rnn1_u, rnn1_bw, rnn1_bu = model.rnn1.weight_ih_l0, model.rnn1.weight_hh_l0, model.rnn1.bias_ih_l0, model.rnn1.bias_hh_l0
rnn1_w_r, rnn1_u_r, rnn1_bw_r, rnn1_bu_r = model.rnn1.weight_ih_l0_reverse, model.rnn1.weight_hh_l0_reverse, model.rnn1.bias_ih_l0_reverse, model.rnn1.bias_hh_l0_reverse
rnn2_w, rnn2_u, rnn2_bw, rnn2_bu = model.rnn2.weight_ih_l0, model.rnn2.weight_hh_l0, model.rnn2.bias_ih_l0, model.rnn2.bias_hh_l0
rnn2_w_r, rnn2_u_r, rnn2_bw_r, rnn2_bu_r = model.rnn2.weight_ih_l0_reverse, model.rnn2.weight_hh_l0_reverse, model.rnn2.bias_ih_l0_reverse, model.rnn2.bias_hh_l0_reverse
rnn3_w, rnn3_u, rnn3_bw, rnn3_bu = model.rnn3.weight_ih_l0, model.rnn3.weight_hh_l0, model.rnn3.bias_ih_l0, model.rnn3.bias_hh_l0
rnn3_w_r, rnn3_u_r, rnn3_bw_r, rnn3_bu_r = model.rnn3.weight_ih_l0_reverse, model.rnn3.weight_hh_l0_reverse, model.rnn3.bias_ih_l0_reverse, model.rnn3.bias_hh_l0_reverse
rnn4_w, rnn4_u, rnn4_bw, rnn4_bu = model.rnn4.weight_ih_l0, model.rnn4.weight_hh_l0, model.rnn4.bias_ih_l0, model.rnn4.bias_hh_l0
rnn4_w_r, rnn4_u_r, rnn4_bw_r, rnn4_bu_r = model.rnn4.weight_ih_l0_reverse, model.rnn4.weight_hh_l0_reverse, model.rnn4.bias_ih_l0_reverse, model.rnn4.bias_hh_l0_reverse

In [6]:
print ('CNN 1 : ', 'W = 32x3x10,', 'b = 32,', '960 + 32 = 992')
print ('CNN 2 : ', 'W = 64x32x7,', 'b = 64,', '14336 + 64 = 14400')
print ('CNN 3 : ', 'W = 128x64x5,', 'b = 128,', '40960 + 128 = 41088')
print ('CNN 4 : ', 'W = 256x128x3,', 'b = 256,', '98304 + 256 = 98560')

CNN 1 :  W = 32x3x10, b = 32, 960 + 32 = 992
CNN 2 :  W = 64x32x7, b = 64, 14336 + 64 = 14400
CNN 3 :  W = 128x64x5, b = 128, 40960 + 128 = 41088
CNN 4 :  W = 256x128x3, b = 256, 98304 + 256 = 98560


In [7]:
print ('DCNN 1 : ', 'W = 96x1x10,', 'b = 1,', '960 + 1 = 961')
print ('DCNN 2 : ', 'W = 192x32x7,', 'b = 32,', '43008 + 32 = 43040')
print ('DCNN 3 : ', 'W = 384x64x5,', 'b = 64,', '122880 + 64 = 122944')
print ('DCNN 4 : ', 'W = 768x128x3,', 'b = 128,', '294912 + 128 = 295040')

DCNN 1 :  W = 96x1x10, b = 1, 960 + 1 = 961
DCNN 2 :  W = 192x32x7, b = 32, 43008 + 32 = 43040
DCNN 3 :  W = 384x64x5, b = 64, 122880 + 64 = 122944
DCNN 4 :  W = 768x128x3, b = 128, 294912 + 128 = 295040


In [8]:
print ('RNN 1 : ', 'W = 32x32 + 32x32,', 'b = 32 + 32,', 'U = 32x32 + 32x32,', 'b_u = 32 + 32,', '4096 + 128 = 4224')
print ('RNN 2 : ', 'W = 64x64 + 64x64,', 'b = 64 + 64,', 'U = 64x64 + 64x64,', 'b_u = 64 + 64,', '16384 + 256 = 16640')
print ('RNN 3 : ', 'W = 128x128 + 128x128,', 'b = 128 + 128,', 'U = 128x128 + 128x128,', 'b_u = 128 + 128,', '65536 + 512 = 66048')
print ('RNN 4 : ', 'W = 256x256 + 256x256,', 'b = 256 + 256,', 'U = 256x256 + 256x256,', 'b_u = 256 + 256,', '262144 + 1024 = 263168')


RNN 1 :  W = 32x32 + 32x32, b = 32 + 32, U = 32x32 + 32x32, b_u = 32 + 32, 4096 + 128 = 4224
RNN 2 :  W = 64x64 + 64x64, b = 64 + 64, U = 64x64 + 64x64, b_u = 64 + 64, 16384 + 256 = 16640
RNN 3 :  W = 128x128 + 128x128, b = 128 + 128, U = 128x128 + 128x128, b_u = 128 + 128, 65536 + 512 = 66048
RNN 4 :  W = 256x256 + 256x256, b = 256 + 256, U = 256x256 + 256x256, b_u = 256 + 256, 262144 + 1024 = 263168


## Plot rnn matrices

In [9]:
model = torch.load('../../weights/modelNewLawDQSynthData_encdecBilstmSkip_act_relu_stride_1_window_100_inpQuants_Voltage1,Voltage2,Speed_outQuants_Torque_lr_0.0001_batchSize_8000_epochs_2000.pt')
model.eval()

EncDecBiLSTMSkipNet(
  (cnn1): Conv1d(3, 32, kernel_size=(10,), stride=(1,))
  (cnn2): Conv1d(32, 64, kernel_size=(7,), stride=(1,))
  (cnn3): Conv1d(64, 128, kernel_size=(5,), stride=(1,))
  (cnn4): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
  (rnn1): RNN(32, 32, batch_first=True, bidirectional=True)
  (rnn2): RNN(64, 64, batch_first=True, bidirectional=True)
  (rnn3): RNN(128, 128, batch_first=True, bidirectional=True)
  (rnn4): RNN(256, 256, batch_first=True, bidirectional=True)
  (dcnn4): ConvTranspose1d(768, 128, kernel_size=(3,), stride=(1,))
  (dcnn3): ConvTranspose1d(384, 64, kernel_size=(5,), stride=(1,))
  (dcnn2): ConvTranspose1d(192, 32, kernel_size=(7,), stride=(1,))
  (dcnn1): ConvTranspose1d(96, 1, kernel_size=(10,), stride=(1,))
)

In [10]:
cnn1_w, cnn1_b = model.cnn1.weight, model.cnn1.bias
cnn2_w, cnn2_b = model.cnn2.weight, model.cnn2.bias
cnn3_w, cnn3_b = model.cnn3.weight, model.cnn3.bias
cnn4_w, cnn4_b = model.cnn4.weight, model.cnn4.bias

dcnn1_w, dcnn1_b = model.dcnn1.weight, model.dcnn1.bias
dcnn2_w, dcnn2_b = model.dcnn2.weight, model.dcnn2.bias
dcnn3_w, dcnn3_b = model.dcnn3.weight, model.dcnn3.bias
dcnn4_w, dcnn4_b = model.dcnn4.weight, model.dcnn4.bias

rnn1_w, rnn1_u, rnn1_bw, rnn1_bu = model.rnn1.weight_ih_l0, model.rnn1.weight_hh_l0, model.rnn1.bias_ih_l0, model.rnn1.bias_hh_l0
rnn1_w_r, rnn1_u_r, rnn1_bw_r, rnn1_bu_r = model.rnn1.weight_ih_l0_reverse, model.rnn1.weight_hh_l0_reverse, model.rnn1.bias_ih_l0_reverse, model.rnn1.bias_hh_l0_reverse
rnn2_w, rnn2_u, rnn2_bw, rnn2_bu = model.rnn2.weight_ih_l0, model.rnn2.weight_hh_l0, model.rnn2.bias_ih_l0, model.rnn2.bias_hh_l0
rnn2_w_r, rnn2_u_r, rnn2_bw_r, rnn2_bu_r = model.rnn2.weight_ih_l0_reverse, model.rnn2.weight_hh_l0_reverse, model.rnn2.bias_ih_l0_reverse, model.rnn2.bias_hh_l0_reverse
rnn3_w, rnn3_u, rnn3_bw, rnn3_bu = model.rnn3.weight_ih_l0, model.rnn3.weight_hh_l0, model.rnn3.bias_ih_l0, model.rnn3.bias_hh_l0
rnn3_w_r, rnn3_u_r, rnn3_bw_r, rnn3_bu_r = model.rnn3.weight_ih_l0_reverse, model.rnn3.weight_hh_l0_reverse, model.rnn3.bias_ih_l0_reverse, model.rnn3.bias_hh_l0_reverse
rnn4_w, rnn4_u, rnn4_bw, rnn4_bu = model.rnn4.weight_ih_l0, model.rnn4.weight_hh_l0, model.rnn4.bias_ih_l0, model.rnn4.bias_hh_l0
rnn4_w_r, rnn4_u_r, rnn4_bw_r, rnn4_bu_r = model.rnn4.weight_ih_l0_reverse, model.rnn4.weight_hh_l0_reverse, model.rnn4.bias_ih_l0_reverse, model.rnn4.bias_hh_l0_reverse

In [11]:
def plot_rnn(w):
    w = w.data.cpu().numpy()
    plt.imshow(w)

## Find filters to prune

In [12]:
def compute_filter_scores(w):
    filters_scores = {}

    cnn_w_npy = w.data.cpu().numpy()
    for i in range(cnn_w_npy.shape[0]):
        score = 0
        for j in range(cnn_w_npy.shape[1]):
            score += np.linalg.norm(cnn_w_npy[i,j,:], ord=1)
        filters_scores[i] = score

    sorted_filters_scores = sorted(filters_scores.items(), key=operator.itemgetter(1))
    sorted_filters_scores.reverse()
    
    return sorted_filters_scores

In [13]:
compute_filter_scores(cnn1_w)

[(22, 3.355632483959198),
 (23, 3.2577165365219116),
 (27, 3.1860647201538086),
 (20, 3.115381360054016),
 (30, 3.09181809425354),
 (7, 3.079788625240326),
 (10, 3.074093818664551),
 (2, 3.0070873498916626),
 (4, 2.998003304004669),
 (15, 2.909471571445465),
 (9, 2.8861640095710754),
 (3, 2.869348883628845),
 (1, 2.8606576919555664),
 (18, 2.8593611121177673),
 (25, 2.8182698488235474),
 (0, 2.8087316155433655),
 (29, 2.8000152111053467),
 (31, 2.7810418605804443),
 (21, 2.7060452699661255),
 (16, 2.642850637435913),
 (28, 2.6322097182273865),
 (8, 2.6197911500930786),
 (19, 2.611757218837738),
 (14, 2.5517998933792114),
 (12, 2.5032225847244263),
 (26, 2.494011342525482),
 (17, 2.435724377632141),
 (24, 2.294153928756714),
 (11, 2.2763158082962036),
 (5, 2.2667396664619446),
 (6, 2.1737363934516907),
 (13, 2.0068514347076416)]